In [ ]:
import pandas as pd

In [ ]:
data=pd.read_csv("data.csv")

In [ ]:
bad=data['Bad coding Practice']

In [ ]:
type(bad)

pandas.core.series.Series

In [ ]:
data

,Coding Problem,Bad coding Practice,correction
0,Inline style,"<p style=""color: red; font-size: 16px;"">This i...",.styled-text {text-align: center;font-size: 18...
1,Inline style,"<span style=""font-weight: bold; color: blue;"">...",.bold-blue-text {font-weight: bold; color: blue;}
2,Inline style,"<div style=""background-color: #ffcc00; padding...",.yellow-background {background-color: #ffcc00;...
3,Inline style,"<h2 style=""text-decoration: underline; color: ...",.underlined-green-heading {text-decoration: un...
4,Inline style,"<a style=""text-decoration: none; color: #9900c...",.purple-link {text-decoration: none; color: #9...
...,...,...,...
95,Overreliance on Deprecated Attributes:,"<img align=""left"" hspace=""10"" vspace=""5"" src=""...",.styled-image {float: left; margin-right: 10px...
96,Overreliance on Deprecated Attributes:,<strike><i>Strikethrough and italicized text</...,.strikethrough-italic-text {text-decoration: l...
97,Overreliance on Deprecated Attributes:,"<marquee behavior=""scroll"" direction=""left"">Sc...",.scrolling-text {animation: marquee-scroll-lef...
98,Overreliance on Deprecated Attributes:,<b>Bold text using deprecated bold tag.</b>,.bold-text {font-weight: bold;}


In [ ]:
bad2=bad.values.tolist()

In [ ]:
good=data['correction'].values.tolist()

In [ ]:
bad_html = bad2
good_html = good

In [ ]:
!pip install tensorflow torch transformers
!pip install sentencepiece==0.1.94

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 63.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.2/311.2 kB 23.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 85.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 69.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 30.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.5/507.5 kB 3.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for sentencepiece: filename=sentencepiece-0.1.94-cp310-cp310-linux_x86_64.whl size=1426797 sha256=da15be4c1b603ce2d78a9ffeda4c19927b0edb29630032b9737e8afc29c33f09
  Stored in directory: /root/.cache/pip/wheels/09/a1/54/a3196ca6f241737de6ae3dcdf00a80383f9dd5e5f7dc78a97e
Successfully built sentencepiece


In [ ]:
!pip install html.parser

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 1.7 MB/s eta 0:00:00
  Created wheel for html.parser: filename=html_parser-0.2-py3-none-any.whl size=1287 sha256=1359592770cbeadbfe11a1ad4d1faf311043fe9f0ba073afc44dbddb1428ce68
  Stored in directory: /root/.cache/pip/wheels/86/72/1e/d46366a3a2a3843b8b19f0c3c4de3b850247f22b61966de114
Successfully built html.parser


In [ ]:
def preprocess_html(html_text, reverse=False):
    html_text = html_text.replace('<', ' [LESS_THAN] ')
    html_text = html_text.replace('>', ' [GREATER_THAN] ')
    if reverse:
      html_text = html_text.replace('[LESS_THAN]', '<')
      html_text = html_text.replace('[GREATER_THAN]', '>')
      html_text = html_text.replace('GREATER_THAN]', '>')
      html_text = html_text.replace('LESS_THAN]', '<')
    return html_text

In [ ]:
from html.parser import HTMLParser
from transformers import T5Tokenizer

class HTMLAwareTokenizer(HTMLParser):
    def __init__(self):
        super().__init__()
        self.tokens = []

    def handle_starttag(self, tag, attrs):
        self.tokens.append(f'<{tag}>')

    def handle_endtag(self, tag):
        self.tokens.append(f'</{tag}>')

    def handle_data(self, data):
        self.tokens.extend(data.split())

def tokenize_html_aware(html_text,tokenizer):

    lis=[]
    html_tokenizer = HTMLAwareTokenizer()
    for i in html_text:
      html_tokenizer.feed(i)
      tokens = html_tokenizer.tokens
      cleaned_text = ' '.join(tokens)
      cleaned_text=preprocess_html(cleaned_text)
      lis.append(cleaned_text)
    tokens = tokenizer(lis, return_tensors="pt",padding=True,truncation=True)

    return tokens

In [ ]:

from transformers import T5ForConditionalGeneration
from torch.utils.data import Dataset, DataLoader

class HtmlDataset(Dataset):
    def __init__(self,bad,good):
      self.bad=bad
      self.good=good
    def __len__(self):
        return len(self.bad)

    def __getitem__(self,idx):
        return {"text":bad[idx], "label":good[idx]}


train_dataset =HtmlDataset(bad_html,good_html)

model = T5ForConditionalGeneration.from_pretrained("t5-base").to('cuda:0')
tokenizer = T5Tokenizer.from_pretrained("t5-base")

def train(model, tokenizer, dataset, epochs=3):

    pass

train(model, tokenizer, train_dataset)



/usr/local/lib/python3.10/dist-packages/transformers/models/t5/tokenization_t5.py:240: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(
You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the

In [ ]:
from transformers import T5Tokenizer
tokenizer = T5Tokenizer.from_pretrained("t5-base")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
import torch
from torch.optim import Adam
from torch.nn.functional import cross_entropy

def train(model, tokenizer, train_dataset, epochs=3, learning_rate=5e-5):
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    model.to(device)
    model.train()

    optimizer = Adam(model.parameters(), lr=learning_rate)

    for epoch in range(epochs):
        total_loss = 0.0

        for batch in DataLoader(train_dataset, batch_size=2, shuffle=True, pin_memory=True):

            inputs = tokenize_html_aware(batch['text'],tokenizer)

            labels = tokenize_html_aware(batch['label'],tokenizer)

            labels.to(device)
            inputs.to(device)

            optimizer.zero_grad()

            outputs = model(**inputs, labels=labels["input_ids"])
            loss = outputs.loss
            total_loss += loss.item()

            loss.backward()
            optimizer.step()

        avg_loss = total_loss / len(train_dataset)
        print(f"Epoch {epoch + 1}/{epochs}, Loss: {avg_loss}")

    model.save_pretrained("fine_tuned_model")



In [ ]:
train(model, tokenizer, train_dataset, epochs=50, learning_rate=0.001)

Epoch 1/50, Loss: 0.11685561444610357
Epoch 2/50, Loss: 0.13668889086693525
Epoch 3/50, Loss: 0.08248032733798027
Epoch 4/50, Loss: 0.07315214335918427
Epoch 5/50, Loss: 0.06998946206644177
Epoch 6/50, Loss: 0.05546237178146839
Epoch 7/50, Loss: 0.046974795665591956
Epoch 8/50, Loss: 0.04782466156408191
Epoch 9/50, Loss: 0.056080708084627985
Epoch 10/50, Loss: 0.0462332333996892
Epoch 11/50, Loss: 0.041535754334181545
Epoch 12/50, Loss: 0.029181362974923103
Epoch 13/50, Loss: 0.027771404865197837
Epoch 14/50, Loss: 0.03410055966116488
Epoch 15/50, Loss: 0.030883202515542506
Epoch 16/50, Loss: 0.029618938425555824
Epoch 17/50, Loss: 0.02683588753454387
Epoch 18/50, Loss: 0.03068915705662221
Epoch 19/50, Loss: 0.026431082570925354
Epoch 20/50, Loss: 0.0206684273481369
Epoch 21/50, Loss: 0.04338015899993479
Epoch 22/50, Loss: 0.028874763071071355
Epoch 23/50, Loss: 0.02748593283817172
Epoch 24/50, Loss: 0.026854710085317493
Epoch 25/50, Loss: 0.025939097390510143
Epoch 26/50, Loss: 0.0244

In [ ]:
next(model.parameters()).device

device(type='cuda', index=0)

In [ ]:
def generate_correction(model, tokenizer, input_text):
    input_ids = tokenizer.encode(input_text, return_tensors="pt", max_length=512,truncation=True).to('cuda:0')
    output_ids = model.generate(input_ids, max_length=100, num_beams=5, length_penalty=0.6)
    corrected_text = tokenizer.decode(output_ids[0], skip_special_tokens=True)
    return corrected_text

# Testing
bad_html_example = '<h3>unclosed'
bad_html_example=preprocess_html(bad_html_example)
correction = generate_correction(model, tokenizer, bad_html_example)
correction=preprocess_html(correction, reverse=True)
print(correction)


< h3 > Properly closed heading < /h3 >
